In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

from pandarallel import pandarallel
pandarallel.initialize()

from IPython.display import HTML
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
out_dir_path = '/mnt/c/Users/uditr/OneDrive/Projects/BigDataBowl/output/{}'

In [17]:
vals = []

for game_id in tqdm(sorted(os.listdir(out_dir_path.format('')))):
    for play_id in tqdm(sorted(os.listdir(out_dir_path.format(f'{game_id}')))):
        for frame_id in sorted(os.listdir(out_dir_path.format(f'{game_id}/{play_id}'))):
            with open(out_dir_path.format(f'{game_id}/{play_id}/{frame_id}'), 'rb') as f:
                all = np.load(f, allow_pickle=True)
                eppa_ind = all['eppa_ind']
                player_info = all['players_ind_info']
                eppa = np.sum(eppa_ind, axis=-1, where=player_info[:, 1]=='OFF')
                vals.append((game_id, play_id, frame_id, eppa))

len(vals)

100%|██████████| 47/47 [11:59<00:00, 15.30s/it]


2714

In [25]:
eppa_sums = list(map(lambda x: x[3].sum(), vals))

In [33]:
df = pd.DataFrame(list(map(lambda x: (x[0], x[1], int(x[2][:-4]),x[3].sum()), vals)), columns=['gameId', 'playId', 'frameId', 'eppa_sum'])
df.head(10)

,gameId,playId,frameId,eppa_sum
0,2018090600,3257,11,0.271333
1,2018090600,3257,12,0.275441
2,2018090600,3257,13,0.276110
3,2018090600,3257,14,0.290340
4,2018090600,3257,15,0.315350
5,2018090600,3257,16,0.346779
6,2018090600,3257,17,0.360562
7,2018090600,3257,18,0.381215
8,2018090600,3257,19,0.409774
9,2018090600,3257,20,0.429509


In [35]:
df[['eppa_sum']].describe()

,eppa_sum
count,2714.000000
mean,0.630291
std,0.321244
min,0.078087
25%,0.391682
50%,0.589892
75%,0.844659
max,1.669557


In [46]:
df.sort_values('eppa_sum', ascending=False)

,gameId,playId,frameId,eppa_sum
2446,2018093005,2703,24,1.669557
2447,2018093005,2703,25,1.660921
2442,2018093005,2703,20,1.660528
2441,2018093005,2703,19,1.658250
2443,2018093005,2703,21,1.657809
...,...,...,...,...
607,2018091001,3904,13,0.094118
606,2018091001,3904,12,0.089882
605,2018091001,3904,11,0.087769
2295,2018093002,1058,12,0.082920
